# Rohaan Advani - N19682109 - rna3535

# The SVM vs. Logistic Regression Showdown

In this lab, you will practice working with non-linear kernels combined with logistic regression and SVM classifiers. The goal is to compare these commonly used techniques. Which comes out on top in terms of accuracy? Runtime? Is there much of a difference at all? Is the dominance of the SVM classifier in machine learning pedagogy justified?

## Loading the Data

First, we load all the packages we'll need.

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import pairwise_kernels
import scipy
from sklearn import svm, linear_model
from sklearn.model_selection import GridSearchCV
import time

Again we download the data from the Tensorflow package, which you will need to install.  You can get the data from other sources as well.

In the Tensorflow dataset, the training and test data are represented as arrays:

     Xtr.shape = 60000 x 28 x 28
     Xts.shape = 10000 x 28 x 28
     
The test data consists of `60000` images of size `28 x 28` pixels; the test data consists of `10000` images.

In [2]:
import tensorflow as tf

(Xtr_raw,ytr),(Xts_raw,yts) = tf.keras.datasets.mnist.load_data()

print('Xtr shape: %s' % str(Xtr_raw.shape))
print('Xts shape: %s' % str(Xts_raw.shape))

ntr = Xtr_raw.shape[0]
nts = Xts_raw.shape[0]
nrow = Xtr_raw.shape[1]
ncol = Xtr_raw.shape[2]

11490434/11490434 [==============================] - 0s 0us/step
Xtr shape: (60000, 28, 28)
Xts shape: (10000, 28, 28)


Each pixel value is from `[0,255]`.  For this lab, it will be convenient to recale the value to -1 to 1 and reshape it as a `ntr x npix` and `nts x npix`.

In [3]:
npix = nrow*ncol
Xtr = Xtr_raw.reshape((ntr,npix))
print(Xtr[1,:])
Xtr = (Xtr/255 - .5)
print(Xtr[1,:])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  51 159 253 159  50   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  48 238 252 252 252 237   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  54 227 253 252 239 233 252  57   6   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  10  60 224 252 253 252 202  84 252
 253 122   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 163 252 252 252 253 252 252  96 189 253 167   

In [4]:
npix = nrow*ncol
Xtr = (Xtr_raw/255 - 0.5)
Xtr = Xtr.reshape((ntr,npix))

Xts = (Xts_raw/255 - 0.5)
Xts = Xts.reshape((nts,npix))

In [5]:
print(Xtr)

[[-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 ...
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]
 [-0.5 -0.5 -0.5 ... -0.5 -0.5 -0.5]]


For this lab we're only going to use a fraction of the MNIST data -- otherwise our models will take too much time and memory to run. Using only part of the training data will of course lead to worse results. Given enough computational resources and time, we would ideally be running on the full data set. The follow code creates a new test and train set, with 10000 examples for train and 5000 for test.

In [6]:
ntr1 = 10000
nts1 = 5000
Iperm = np.random.permutation(ntr1)
Xtr1 = Xtr[Iperm[:ntr1],:]
ytr1 = ytr[Iperm[:ntr1]]
Iperm = np.random.permutation(nts1)
Xts1 = Xts[Iperm[:nts1],:]
yts1 = yts[Iperm[:nts1]]

## Problem set up and establishing a baseline

To simplify the problem (and speed things up) we're also going to restrict to binary classification. In particular, let's try to design classifier a that separates the 8's from all other digits.

Create binary 0/1 label vectors `ytr8` and `yts8` which are 1 wherever `ytr1` and `yts1` equal 8, and 0 everywhere else.

In [7]:
ytr8 = np.where(ytr1 == 8, 1, 0)
yts8 = np.where(yts1 == 8, 1, 0)

Most of the digits in the test dataset aren't equal to 8. So if we simply guess 0 for every image in `Xts`, we might expect to get classification accuracy around 90%. Our goal should be to significantly beat this **baseline**.

Formally, write a few lines of code to check what test error would be achieved by the all zeros classifier.

In [8]:
# Calculate the accuracy of the all-zeros classifier
pred_zeros = np.zeros_like(yts8)
acc = np.mean(pred_zeros == yts8)
print('Accuaracy = {0:f}%'.format(acc * 100))

Accuaracy = 90.220000%


As a second baseline, let's see how we do with standard (non-kernel) logistic regression. As in the MNIST demo, you can use `scikit-learn`'s built in function `linear_model.LogisticRegression` to fit the model and compute the accuracy. Use no regularization and the `lbfgs` solver. You should acheive an improvement to around 93-95%.

In [9]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(penalty='none', solver='lbfgs')
logistic_model.fit(Xtr1, ytr8)

pred_logistic = logistic_model.predict(Xts1)
acc = np.mean(pred_logistic == yts8)
print('Logistic Regression Accuracy = {0:f}%'.format(acc * 100))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(


Logistic Regression Accuracy = 94.100000%


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Kernel Logistic Regression

To improve on this baseline performance, let's try using the logistic regression classifier with a *non-linear* kernel. Recall from class that any non-linear kernel similarity function $k(\vec{w},\vec{z})$ is equal to $\phi(\vec{w})^T\phi(\vec{z})$ for some feature transformation $\phi$. However, we typically do not need to compute this feature tranformation explicitly: instead we can work directly with the kernel gram matrix $K \in \mathbb{R}^{n\times n}$. Recall that $K_{i,j} = k(\vec{x}_i,\vec{x}_j)$ where $\vec{x}_i$ is the $i^\text{th}$ training data point.

For this lab we will be using the radial basis function kernel. For a given scaling factor $\gamma$ this kernel is defined as:
$$
k(\vec{w},\vec{z}) = e^{-\gamma\|\vec{w}-\vec{z}\|_2^2}
$$

In [10]:
def rbf_kernel(w,z,gamma):
    d = w - z
    return np.exp(-gamma*np.sum(d*d))

Construct the kernel matrix `K1` for `Xtr1` with `gamma = .05`.

In [11]:
gamma = 0.05
n_tr1 = Xtr1.shape[0]
K1 = np.zeros((n_tr1, n_tr1))

for i in range(n_tr1):
    for j in range(n_tr1):
        K1[i, j] = rbf_kernel(Xtr1[i], Xtr1[j], gamma)
print(K1)

[[1.         0.00288793 0.01608046 ... 0.00583719 0.01482722 0.01056562]
 [0.00288793 1.         0.00140932 ... 0.00224628 0.00452842 0.00175463]
 [0.01608046 0.00140932 1.         ... 0.01016976 0.03766971 0.00323562]
 ...
 [0.00583719 0.00224628 0.01016976 ... 1.         0.05440837 0.00530254]
 [0.01482722 0.00452842 0.03766971 ... 0.05440837 1.         0.00223219]
 [0.01056562 0.00175463 0.00323562 ... 0.00530254 0.00223219 1.        ]]


If you used a for loop (which is fine) your code might take several minutes to run! Part of the issue is that Python won't know to properly parallize your for loop. For this reason, when constructing kernel matrices it is often faster to us a built-in, carefully optimized function with explicit parallelization. Scikit learn provides such a function through their `metrics` library.

Referring to the documentation here
https://scikit-learn.org/stable/modules/metrics.html#metrics, use this built in function to recreate the same kernel matrix you did above. Store the result at `K`.

In [12]:
from sklearn.metrics.pairwise import pairwise_kernels

gamma = 0.05
K = pairwise_kernels(Xtr1, metric='rbf', gamma=gamma)
print(K)

[[1.         0.00288793 0.01608046 ... 0.00583719 0.01482722 0.01056562]
 [0.00288793 1.         0.00140932 ... 0.00224628 0.00452842 0.00175463]
 [0.01608046 0.00140932 1.         ... 0.01016976 0.03766971 0.00323562]
 ...
 [0.00583719 0.00224628 0.01016976 ... 1.         0.05440837 0.00530254]
 [0.01482722 0.00452842 0.03766971 ... 0.05440837 1.         0.00223219]
 [0.01056562 0.00175463 0.00323562 ... 0.00530254 0.00223219 1.        ]]


Check that you used the function correctly by writing code to confirm that `K = K1`, or at least that the two are equal up to very small differences (which could arise due to numerical precision issues). Try to do this **without a for loop** for full credit. You will get partial credit if you use a for loop.

In [13]:
diff_matrix = np.abs(K - K1)

# Check if the maximum absolute difference is below a small threshold
threshold = 1e-6
max_diff = np.max(diff_matrix)

if max_diff < threshold:
    print("The kernel matrices K and K1 are equal up to very small differences.")
else:
    print("The kernel matrices K and K1 are not equal up to the specified threshold.")

The kernel matrices K and K1 are equal up to very small differences.


When using a non-linear kernel, it is important to check that you have chosen reasonable parameters (in our case the only parameter is `gamma`). We typically do not want $k(\vec{x}_i,\vec{x}_j)$ to be either negligably small, or very large for all $i\neq j$ in our data set, or we won't be able to learn anything. For the RBF kernel this means that, for any $\vec{x}_i$ we don't want $k(\vec{x}_i,\vec{x}_j)$ very close to 1 (e.g. .9999) for all $j$, or very close to $0$ (e.g. 1e-8) for all $j$.

Let's just check that we're in good shape for the first data vector $\vec{x}_0$. Do so by printing out the 10 largest and 10 smallest values of $k(\vec{x}_0,\vec{x}_j)$ for $j\neq 0$. Note that we always have $k(\vec{x}_0,\vec{x}_0) = 1$ for the RBF kernel.

In [14]:
# Find the kernel values for the first data vector x_0
k_x0 = K[0, 1:]  # Exclude the diagonal element k(x_0, x_0)

# Find the indices of the 10 largest and 10 smallest values
largest_indices = np.argsort(k_x0)[-10:]
smallest_indices = np.argsort(k_x0)[:10]
print('Maximum similarities: \n', k_x0[largest_indices])
print('Minimum similarities: \n', k_x0[smallest_indices])

Maximum similarities: 
 [0.58036747 0.58607181 0.59000335 0.59983242 0.60170194 0.60711112
 0.61189479 0.61683804 0.65094391 0.72168309]
Minimum similarities: 
 [4.49645784e-05 4.53226708e-05 4.89093250e-05 5.85876553e-05
 5.94204585e-05 6.39691039e-05 6.48993585e-05 6.67168215e-05
 7.03508720e-05 7.25316246e-05]


### Implementation
Maybe surprisingly Scikit learn does not have an implementation for kernel logistic regression. So we have to implement our own!

Write a function function `log_fit` that minimizes the $\ell_2$-regularized logisitic regression loss:
$$
L(\boldsymbol{\alpha}) =\sum_{i=1}^n (1-y_i)(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\vec{\alpha}) - \log(h(\phi(\mathbf{x}_i)^T\phi(\mathbf{X})^T\boldsymbol{\alpha}) + \lambda \|\phi(\mathbf{X})^T\boldsymbol{\alpha}\|_2^2.
$$
As input it takes an $n\times n$ kernel matrix $K$ for the training data, an $n$ length vector `y` of binary class labels, and regularization parameter `lamb`.

To implement your function you can either use your own implmentation of gradient descent or used a built in minimizer from `scipy.optimize.minimize`. I recommend the later approach: as we saw in the last lab, gradient descent can converge slowly for this objective.

In [15]:
from scipy.optimize import minimize

def log_fit(K,y,lamb,nit=1000):

  def objective(alpha):
    z = K @ alpha
    h = 1 / (1 + np.exp(-z))
    loss = np.sum((1 - y) * z - np.log(h)) + lamb * alpha.T @ K @ alpha
    return loss

  def gradient(alpha):
    z = K @ alpha
    h = 1 / (1 + np.exp(-z))
    grad = K.T @ (h - y) + 2 * lamb * K @ alpha
    return grad

  alpha = np.zeros(K.shape[0])

  result = minimize(objective, alpha, method='L-BFGS-B', jac=gradient, options={'maxiter': nit})
  return result.x

Use the `log_fit` function defined above to find parameters `alpha` for the kernel logistic regression model using `lamb = 0` and `K` as constructed above (with `gamma = .05`).

In [16]:
lamb = 0
alpha = log_fit(K, ytr8, lamb)
print(alpha)

[ 3.93429526 -3.09204208 -1.66931489 ... 29.24373634  9.06654502
  4.28038612]


Suppose we have a test dataset with $m$ examples $\vec{w}_1,\ldots, \vec{w}_m$. Once we obtain a coefficient vector $\alpha$, making predictions for any $\vec{w}_j$ in the test set requires computing:
$$
{y}_{j} = \sum_{i=1}^n \alpha_i \cdot k(\vec{w}_{j}, \vec{x}_i).
$$
where $\vec{x}_1, \ldots \vec{x}_n$ are our training data vectors. We classify $\vec{w}_{j}$ in class 0 if ${y}_{j} \leq 0$ and in class 1 if ${y}_{j} > 0$.

This computation can be rewritten in matrix form as follows:
$$
\vec{y}_{test} = K_{test}\vec{\alpha},
$$
where $\vec{y}_{text}$ is an $m$ length vector and $K_{test}$ is a $m\times n$ matrix whose $(j,i)$ entry is equal to $k(\vec{w}_{j}, \vec{x}_i)$. We classify $\vec{w}_{j}$ in class 0 if $\vec{y}_{test}[j] \leq 0$ and in class 1 if $\vec{y}_{test}[j] > 0$.


Use the `pairwise_kernels` function to construct $K_{test}$. Then make predictions for the test set and evaluate the accuracy of our kernel logistic regression classifier. You should see a pretty substantial lift in accuracy to around $97\%$

In [17]:
Ktest = pairwise_kernels(Xts1, Xtr1, metric='rbf', gamma=gamma)

In [18]:
yhat = np.sign(Ktest.dot(alpha))

# Convert predictions to 0/1 labels
yhat_binary = (yhat > 0).astype(int)
acc = np.mean(yhat_binary == yts8)
print('Test Accuracy = {0:f}%'.format(acc * 100))

Test Accuracy = 97.980000%


## Kernel Support Vector Machine

The goal of this lab is to compare Kernel Logistic Regression to Kernel Support Vector machines. Following `demo_mnist_svm.ipynb` create and train an SVM classifier on `Xtr1` and `ytr8` using an RBF kernel with `gamma = .05` (the same value we used for logistic regression aboe). Use margin parameter `C = 10`.

In [19]:
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', gamma=0.05, C=10)
svm_model.fit(Xtr1, ytr8)

SVC(C=10, gamma=0.05)

Calculate and print the accuracy of the SVM classifier. You should obtain a similar result as for logistic regression: something close to $97\%$ accuracy.

In [20]:
ysvm = svm_model.predict(Xts1)
acc = np.mean(ysvm == yts8)
print('SVM Accuracy = {0:f}%'.format(acc * 100))

SVM Accuracy = 98.020000%


## The Showdown

Both SVM classifiers and kernel logisitic regression require tuning parameters to obtain the best possible result. In our setting we will stick with an RBF kernel (although this could be tuned). So we only consider tuning the kernel width parameter `gamma`, as well as the regularization parameter `lamb` for logistic regression, and the margin parameter `C` for SVM. We will choose parameters using for-loops and train-test cross validation.

Train a logistic regression classifier with **all combinations** of the parameters included below in vectors `gamma` and `lamb`. For each setting of parameters, compute and print:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

For computing runtime you might want to use the `time()` function from the `time` library, which we already imported ealier.

In [21]:
gamma = [.1, .05,.02,.01,.005]
lamb = [0,1e-6,1e-4,1e-2]
for g in gamma:
    for l in lamb:
        start_time = time.time()

        # Compute the kernel matrix
        K = pairwise_kernels(Xtr1, metric='rbf', gamma=g)

        # Train the logistic regression classifier
        alpha = log_fit(K, ytr8, l)

        # Compute the kernel matrix for the test dataset
        Ktest = pairwise_kernels(Xts1, Xtr1, metric='rbf', gamma=g)

        yhat = np.sign(Ktest.dot(alpha))
        yhat_binary = (yhat > 0).astype(int)
        acc = np.mean(yhat_binary == yts8)

        end_time = time.time()
        runtime = end_time - start_time

        print("Gamma = {}, Lambda = {}, Test Accuracy = {:.2f}%, Runtime = {:.4f} sec".format(g,l,acc*100,runtime))

Gamma = 0.1, Lambda = 0, Test Accuracy = 96.32%, Runtime = 38.3639 sec
Gamma = 0.1, Lambda = 1e-06, Test Accuracy = 96.58%, Runtime = 96.7666 sec
Gamma = 0.1, Lambda = 0.0001, Test Accuracy = 96.52%, Runtime = 129.7021 sec
Gamma = 0.1, Lambda = 0.01, Test Accuracy = 96.28%, Runtime = 131.6690 sec
Gamma = 0.05, Lambda = 0, Test Accuracy = 97.98%, Runtime = 64.0214 sec
Gamma = 0.05, Lambda = 1e-06, Test Accuracy = 98.02%, Runtime = 291.3439 sec
Gamma = 0.05, Lambda = 0.0001, Test Accuracy = 97.80%, Runtime = 611.8256 sec
Gamma = 0.05, Lambda = 0.01, Test Accuracy = 97.42%, Runtime = 489.3060 sec


<ipython-input-15-7ab5a0460b08>:7: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-z))
<ipython-input-15-7ab5a0460b08>:8: RuntimeWarning: divide by zero encountered in log
  loss = np.sum((1 - y) * z - np.log(h)) + lamb * alpha.T @ K @ alpha
<ipython-input-15-7ab5a0460b08>:13: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-z))


Gamma = 0.02, Lambda = 0, Test Accuracy = 98.14%, Runtime = 319.7423 sec
Gamma = 0.02, Lambda = 1e-06, Test Accuracy = 98.44%, Runtime = 1206.0349 sec
Gamma = 0.02, Lambda = 0.0001, Test Accuracy = 98.38%, Runtime = 1186.1330 sec
Gamma = 0.02, Lambda = 0.01, Test Accuracy = 98.24%, Runtime = 1177.5964 sec


<ipython-input-15-7ab5a0460b08>:7: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-z))
<ipython-input-15-7ab5a0460b08>:8: RuntimeWarning: divide by zero encountered in log
  loss = np.sum((1 - y) * z - np.log(h)) + lamb * alpha.T @ K @ alpha
<ipython-input-15-7ab5a0460b08>:13: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-z))


Gamma = 0.01, Lambda = 0, Test Accuracy = 98.06%, Runtime = 1073.7925 sec
Gamma = 0.01, Lambda = 1e-06, Test Accuracy = 97.88%, Runtime = 1202.9670 sec
Gamma = 0.01, Lambda = 0.0001, Test Accuracy = 98.10%, Runtime = 1182.4242 sec
Gamma = 0.01, Lambda = 0.01, Test Accuracy = 98.28%, Runtime = 1180.6994 sec
Gamma = 0.005, Lambda = 0, Test Accuracy = 97.66%, Runtime = 1182.6439 sec
Gamma = 0.005, Lambda = 1e-06, Test Accuracy = 97.54%, Runtime = 1237.8004 sec
Gamma = 0.005, Lambda = 0.0001, Test Accuracy = 97.76%, Runtime = 920.9669 sec
Gamma = 0.005, Lambda = 0.01, Test Accuracy = 97.36%, Runtime = 762.3987 sec


TODO: What was the best test error achieved, and what setting of parameters achieved this error? Was the kernel logistic regression classifier more sensitive to changes in `gamma` or `lamb`? Discuss in 1-3 short sentences below.

The best test error is that with an accuracy of 98.44 %. The parameters that achieved this were Gamma = 0.02 and Lambda = 1e-06. The kernel logistic regression classifier is more sensitive to changes in gamma.

Gamma controls the width of the kernel, affecting the smoothness and flexibility of the decision boundary. A higher gamma has a complex decision boundary, potentially leading to overfitting, a lower gamma may underfit the data. Lambda controls regularization strength, balancing the fit to the training data and the complexity of the model. Low lambda leads to overfitting, while high lambda may leads underfitting by penalizing the model complexity.

Now let's do the same thing for the kernel Support Vector Classifier. Train an SVM classifier with **all combinations** of the parameters included below in vectors `gamma` and `C`. For each setting of parameters, compute:
* the test error obtained
* the total runtime of classification in seconds (including training time and prediction time)

In [22]:
gamma = [.1, .05,.02,.01,.005]
C = [.01,.1,1,10]
for g in gamma:
    for c in C:
        start_time = time.time()

        # Create and train the SVM classifier
        svm_model = SVC(kernel='rbf', gamma=g, C=c)
        svm_model.fit(Xtr1, ytr8)

        ysvm = svm_model.predict(Xts1)
        acc = np.mean(ysvm == yts8)

        end_time = time.time()
        runtime = end_time - start_time

        print("Gamma = {}, C = {}, Test Accuracy = {:.2f}%, Runtime = {:.4f} sec".format(g,c,acc*100,runtime))

Gamma = 0.1, C = 0.01, Test Accuracy = 90.22%, Runtime = 83.6736 sec
Gamma = 0.1, C = 0.1, Test Accuracy = 90.22%, Runtime = 123.7456 sec
Gamma = 0.1, C = 1, Test Accuracy = 92.82%, Runtime = 147.6938 sec
Gamma = 0.1, C = 10, Test Accuracy = 93.42%, Runtime = 162.7012 sec
Gamma = 0.05, C = 0.01, Test Accuracy = 90.22%, Runtime = 28.7206 sec
Gamma = 0.05, C = 0.1, Test Accuracy = 92.28%, Runtime = 31.0640 sec
Gamma = 0.05, C = 1, Test Accuracy = 97.74%, Runtime = 38.1920 sec
Gamma = 0.05, C = 10, Test Accuracy = 98.02%, Runtime = 39.1549 sec
Gamma = 0.02, C = 0.01, Test Accuracy = 90.22%, Runtime = 20.8629 sec
Gamma = 0.02, C = 0.1, Test Accuracy = 95.12%, Runtime = 17.3904 sec
Gamma = 0.02, C = 1, Test Accuracy = 98.20%, Runtime = 14.2327 sec
Gamma = 0.02, C = 10, Test Accuracy = 98.58%, Runtime = 14.3850 sec
Gamma = 0.01, C = 0.01, Test Accuracy = 90.22%, Runtime = 19.3823 sec
Gamma = 0.01, C = 0.1, Test Accuracy = 94.50%, Runtime = 17.7723 sec
Gamma = 0.01, C = 1, Test Accuracy = 97.

TODO: What was the best test error achieved, and what setting of parameters achieved this error? Which performed better in terms of accuracy, the SVM or logisitic regression classifier? How about in terms of runtime?

The best test error is that with 98.7 % accuracy. The parameters that achieved this were Gamma = 0.01 and C = 10. SVM outperformed logistic regression in terms of accuracy by 0.12. In terms of runtime SVM was much faster than logistic regression.

**NOTE:** For `sklearns`'s built in classifiers, including svm.SVC, there is a function called `GridSearchCV` which can automatically perform hyperparamater tuning for you. The main advantage of the method (as opposed to writing for-loops) is that it supports parallelization, so it can fit with different parameters at the same time. The function also supports automatic $k$-fold cross-validation (instead of simple train/test split).

You might be interested in using this function in the future. If so, please check out the tutorial in the following lab from previous year: https://github.com/sdrangan/introml/blob/master/unit08_svm/lab_emnist_partial.ipynb.